# Multi-Blockchain Wallet in Python

### Importing Libraries

In [1]:
import subprocess
import json
from dotenv import load_dotenv
import os
from constants import *
from bit.network import NetworkAPI
from bit import Key, PrivateKey, PrivateKeyTestnet
from web3 import Web3
from web3.middleware import geth_poa_middleware
from eth_account import Account
load_dotenv('c.env')

True

### Connect to Web3 and load mnemonic

In [2]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))

In [3]:
w3.isConnected()

True

In [4]:
mnemonic = os.getenv('mnemonic')

In [5]:
type(mnemonic)

str

In [6]:
# Create a dictionary object that uses the `derive_wallets` function to derive the selected coins

coins = {"eth", "btc-test", "btc"}
numderive = 3

### Create a function that derives the wallet keys

In [7]:
def derive_wallets(mnemonic, coin, numderive):
    command = './derive -g --mnemonic="{mnemonic}" --coin="btc-test" --cols=path,address,privkey,pubkey --format=json'
    if coin == "eth":
        command = './derive -g --mnemonic="{mnemonic}" --coin="eth" --cols=path,address,privkey,pubkey --format=json'
    if coin == "btc":
        command = './derive -g --mnemonic="{mnemonic}" --coin="btc" --cols=path,address,privkey,pubkey --format=json'
        
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    p_status = p.wait()
    keys = json.loads(output)
    return keys

In [8]:
keys = {}
for coin in coins:
    keys[coin] = derive_wallets(os.getenv("mnemonic"), coin, numderive)

In [9]:
eth_Privatekey = keys["eth"][0]['privkey']
btc_Privatekey = keys["btc-test"][1]['privkey']

print(json.dumps(eth_Privatekey, indent=4, sort_keys=True))
print(json.dumps(btc_Privatekey, indent=4, sort_keys=True))

"0x9d560f1207789c19a9fbb1c4cf6c8f5828d5b2219034cc7362a3705ab20f2305"
"cSHtav3eA6CQrMJEK6Z54MdVroQDysvggcES4BLV4RW1HtCgTdCh"


In [10]:
print(json.dumps(keys, indent=4, sort_keys=True))

{
    "btc": [
        {
            "address": "15Ji2eBZxbjw7kpeU7A2KXRs2k6wHbaESS",
            "path": "m/44'/0'/0'/0/0",
            "privkey": "KzfiLVcitrrvW7jUcwxKbEHSjsXDKq47X5w3r7uh8DJTThmjZD3R",
            "pubkey": "02fecd1c9636ba4663df6d72b2c3959bee27530807af098173938c608e3c34cd18"
        },
        {
            "address": "1BVCTgqVXuVUugbJVYi2SVAn1WivewzPT1",
            "path": "m/44'/0'/0'/0/1",
            "privkey": "KxJrcUcg9MyeGmJF8M3QzQ1d5BwzJxj5ErqwbJ4EkapQBzx9RZtx",
            "pubkey": "029ff90cfd29575b78928a39a22ba490236c26044d80584748a5e88ebea81a7c9f"
        },
        {
            "address": "1HAWgDGTxgd44gb97Hw4KGxuCmnzTPRU4A",
            "path": "m/44'/0'/0'/0/2",
            "privkey": "L4VhtgSRBAQ3zWTd72H5F8MqW1R1E8bTJq3aAxVqzQBrAS5nWnx4",
            "pubkey": "02fac020fed67ceaaa4f8e138a48a2b77334ec7db239f3ecd6584cff1d529938a5"
        },
        {
            "address": "13HZsdcLSeF8bsLfp8ZU5MeeKyHd7qrLE1",
            "path": "m/44'/0'/0'/0/3",
  

### Convert the privkey string in a child key to an account object that bit or web3.py can use to transact.

In [11]:
def priv_key_to_account(coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    elif coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

In [12]:
eth_account = priv_key_to_account(ETH,eth_Privatekey)
btc_account = priv_key_to_account(BTCTEST,btc_Privatekey)

In [13]:
print(eth_account)

In [14]:
print(btc_account)

<PrivateKeyTestnet: mqvz6G9KMPfDH7Ns5egb9gdAr2anxMJxrF>


### create the raw, unsigned transaction that contains all metadata needed to transact

In [15]:
def create_tx(coin,account, recipient, amount):
    if coin == ETH: 
        gasEstimate = w3.eth.estimateGas(
            {"from":eth_account.address, "to":recipient, "value": amount}
        )
        return { 
            "from": eth_account.address,
            "to": recipient,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(eth_account.address)
        }
    elif coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, "btc")])

In [16]:
def send_tx(coin, account, recipient, amount):
    if coin == ETH:
        txn = create_tx(coin, account, recipient, amount)
        signed_tx = eth_account.signTransaction(txn)
        result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        print(result.hex())
        return result.hex()
    
    elif coin == BTCTEST:
        btc_tx = create_tx(coin, account, recipient, amount)
        signed_tx = account.sign_transaction(txn)
        print(signed_tx)
        return NetworkAPI.broadcast_tx_testnet(signed_tx)

In [17]:
btc_account

<PrivateKeyTestnet: mqvz6G9KMPfDH7Ns5egb9gdAr2anxMJxrF>

### Create a function to send tx

In [20]:
# Create BTC transaction
create_tx(BTCTEST, btc_account, "mqvz6G9KMPfDH7Ns5egb9gdAr2anxMJxrF", 0.000000001)

InsufficientFunds: Balance 7392 is less than 23052 (including fee).

In [ ]:
# Send BTC transaction
send_tx(BTCTEST,btc_account,"mz73QdYC3vz9Qsecyn5wfVLZbXJAz3GG43",0.000001)

### ETH transactions (Due to a bug in web3.py, I sent several transactions with MyCrypto from my local private mining blockchain, since the w3.eth.generateGasPrice() function does not work with an empty chain. The node keystore file is used.

In [ ]:
#connecting to HTTP with address pk
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

In [ ]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))

In [ ]:
my_account = "0x503B3648E16a2103Edd7f9A768601c865714B492"
balance=w3.eth.getBalance(my_account)  # my account my crypto
w3.fromWei(balance, "ether")

In [ ]:
w3.isConnected()

In [ ]:
w3.eth.blockNumber

In [ ]:
w3.eth.getBalance("0x503B3648E16a2103Edd7f9A768601c865714B492")

In [ ]:
eth_account.address

In [ ]:
create_tx(ETH, eth_account,"0x503B3648E16a2103Edd7f9A768601c865714B492", 1000)

In [ ]:
send_tx(ETH, eth_account,"0xbFDf092Ea03FC2ff3f9D0a8aFf0F567638209C0F", 1000)